In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import interp1d
from scipy.stats import lognorm

# Set a seed for reproducibility
np.random.seed(1)

# Model parameters
N = 1000  # number of individuals
infusion_dur = 1  # [h]
infusion_amt = 100
infusion_rate = infusion_amt / infusion_dur  # for use in ODEs
n_maintain = 4
maintain_amt = 200
D = np.repeat([infusion_amt, maintain_amt], [1, n_maintain])  # [mg]
tD = np.arange(0, 24 * (n_maintain + 1), 24)  # daily dose in [h]
route = np.repeat(['iv_infusion', 'oral'], [1, n_maintain])

# Sampling times
t_obs = np.array([0.5, 1, 2, 4, 8, 24 + 2, 24 + 8,
                  48 + 2, 48 + 8, 72 + 2, 72 + 8, 96 + 2, 96 + 8])
n_obs = len(t_obs)  # number of samples per ID

# Hyperparameters
fe = {'ka': 0.2, 'Fbio': 0.4, 'V': 20, 'CL': 3}
sd_re = {'ka': 0.1, 'Fbio': 0.5, 'V': 0.4, 'CL': 0.3}
sd_ruv = 0.2  # [mg/L]

# Individual parameters
theta = pd.DataFrame(index=range(N), columns=['ka', 'Fbio', 'V', 'CL'])
for i in range(N):
    theta.loc[i, 'ka'] = lognorm.rvs(s=sd_re['ka'], scale=np.exp(fe['ka']))
    theta.loc[i, 'Fbio'] = 1 / (1 + np.exp(-lognorm.rvs(s=1, scale=np.log(fe['Fbio']/(1-fe['Fbio'])))))  # logit-normal
    theta.loc[i, 'V'] = lognorm.rvs(s=sd_re['V'], scale=np.exp(fe['V']))
    theta.loc[i, 'CL'] = lognorm.rvs(s=sd_re['CL'], scale=np.exp(fe['CL']))

# Forcing function encoding the infusion
infusion_rate_function = interp1d([0, infusion_dur], [infusion_rate, 0], kind='previous', fill_value='extrapolate')

# ODE right-hand side
def rhs(y, t, param):
    infusion = infusion_rate if y[1] > 0 else 0
    dX = [
        -param['ka'] * y[0],  # gut
        -infusion,            # bag
        param['ka'] * param['Fbio'] * y[0] + infusion - param['CL'] / param['V'] * y[2]  # plasma
    ]
    return dX

ValueError: Domain error in arguments. The `scale` parameter must be positive for all distributions, and many distributions have restrictions on shape parameters. Please see the `scipy.stats.lognorm` documentation for details.

In [ ]:

# Encoding repeated dosing via events
event_times = np.concatenate((tD, np.array([infusion_dur] * N)))
event_values = np.concatenate((D, [0] * N))

# Solving ODE and computing model prediction
y_pred = []
for i in range(N):
    theta_i = theta.iloc[i]
    x0 = [0, 0, 0]  # initial conditions: gut, bag, plasma

    # ODE solution: amount; observable: concentration (= amount/volume)
    t_ode = np.sort(np.concatenate((tD, t_obs)))
    x_sol = odeint(rhs, x0, t_ode, args=(theta_i.to_dict(),))
    
    # Concentration is plasma amount divided by volume
    y_pred.append(x_sol[:, 2] / theta_i['V'])

# Create and visualize data
y_pred = np.array(y_pred)
df = pd.DataFrame({
    'ID': np.repeat(np.arange(1, N + 1), n_obs),
    'time': np.tile(t_obs, N),
    'y_obs': y_pred.flatten() + np.random.normal(0, sd_ruv, N * n_obs),
    'dose': np.nan,
    'route': np.nan,
    'duration': np.nan
})

# Add dosing information
dosing_info = pd.DataFrame({
    'ID': np.repeat(np.arange(1, N + 1), n_maintain + 1),
    'time': np.tile(tD, N),
    'y_obs': np.nan,
    'dose': np.repeat(D, N),
    'route': np.repeat(route, N),
    'duration': np.where(route == 'iv_infusion', infusion_dur, np.nan)
})

df = pd.concat([df, dosing_info], ignore_index=True).sort_values(['ID', 'time'])

# Peek at the data
print(df.head(30))

# Visualization
plt.figure(figsize=(12, 6))
for id in range(1, 11):  # just a subset for easier visualization
    subset = df[(df['ID'] == id) & (~df['y_obs'].isna())]
    plt.plot(subset['time'], subset['y_obs'], marker='o', label=f'ID {id}')
plt.axvline(x=tD, color='blue', linestyle='--', label='Dosing Times')
plt.xlabel('Time [h]')
plt.ylabel('Concentration [mg/L]')
plt.legend()
plt.title('Concentration Over Time for Individuals')
plt.show()

# Write to file
df.to_csv("../data/loadingMaintenance.csv", index=False)



In [3]:
import numpy as np
import os

# Generate a random vector of 10 elements
vector = np.random.rand(10)

# Get the current directory
current_directory = r"C:\Users\cesar\Desktop\Projects\FoundationModels\fimodemix\data\state_sde\expressions-pool-desi2\output"

# Define the file path
file_path = os.path.join(current_directory, 'random_vector.txt')

# Save the vector to a text file
with open(file_path, 'w') as f:
    for value in vector:
        f.write(f"{value}\n")

print(f"Vector saved to {file_path}")


Vector saved to C:\Users\cesar\Desktop\Projects\FoundationModels\fimodemix\data\state_sde\expressions-pool-desi2\dimension_1\random_vector.txt
